Name - Keshav Aggarwal  
Entry no. - 2021CSB1104

In [1]:
# import the necessary packages
from imutils import face_utils
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import numpy as np
import imageio
import dlib
import cv2

model = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(model)

Function func_image1:

In [2]:
# Image-1 function for the part-B fo the question
def func_image1 ():
    # load the input image-1, resize it, and convert it to grayscale
    image1 = plt.imread("img1.jpg")
    image1 = cv2.resize(image1, (500, 600))
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    height, width, ch = image1.shape

    # passing the gray version of image in the face detector and shape predictor to get the array of tie points
    rects1 = detector(gray1, 1)
    for (i, rect) in enumerate(rects1):
        pts1 = predictor(gray1, rect)
        pts1 = face_utils.shape_to_np(pts1)

    # adding corner points to tie points so to make Delaunay triangles to full image
    pts1 = np.append(pts1, [[0,0],[0,height],[width,0],[width,height], [0, int(height/2)], [int(width/2), 0], [width, int(height/2)], [int(width/2), height]], axis=0)

    tri1 = Delaunay(pts1)

    return image1, pts1, tri1

Function func_image2: 

In [3]:
# Image-2 function for the part-B of the question
def func_image2 ():
    # load the input image-2, resize it, and convert it to grayscale
    image2 = plt.imread("img2.jpg")
    image2 = cv2.resize(image2, (500,600))
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    height, width, ch = image2.shape

    # passing the gray version of image in the face detector and shape predictor to get the array of tie points
    rects2 = detector(gray2, 1)
    for (i, rect) in enumerate(rects2):
        pts2 = predictor(gray2, rect)
        pts2 = face_utils.shape_to_np(pts2)

    # adding corner points to tie points so to make Delaunay triangles to full image
    pts2 = np.append(pts2, [[0,0],[0,height],[width,0],[width,height], [0, int(height/2)], [int(width/2), 0], [width, int(height/2)], [int(width/2), height]], axis=0)

    tri2 = Delaunay(pts2)

    return image2, pts2, tri2


Function for intermediate image:

In [4]:
# Funtion to get the intermediate image with tie points at ratio = t

def intermediate_img(image, newpts, image_no, pts1, pts2):
	newimage = image.copy()
	newtri = Delaunay(newpts)

	# Affine transformation with the first image to get various triangles
	rows, cols, ch = image.shape
	for j in range(newtri.simplices.shape[0]):
		if image_no == 1:
			tempts1 = np.float32([pts1[newtri.simplices[j, 0]], pts1[newtri.simplices[j, 1]], pts1[newtri.simplices[j, 2]]])
		elif image_no == 2:
			tempts1 = np.float32([pts2[newtri.simplices[j, 0]], pts2[newtri.simplices[j, 1]], pts2[newtri.simplices[j, 2]]])
		tempts2 = np.float32([newpts[newtri.simplices[j, 0]], newpts[newtri.simplices[j, 1]], newpts[newtri.simplices[j, 2]]])
		tempimg = cv2.getAffineTransform(tempts1, tempts2)
		tempimg = cv2.warpAffine(image, tempimg, (cols, rows))

		pts = np.int32(tempts2)

		mask = np.zeros(tempimg.shape[:2], np.uint8)

		cv2.fillPoly(mask, pts=[pts], color=(255, 255, 255))
		tempimg = cv2.bitwise_and(tempimg, tempimg, mask=mask)
		mask[mask == 255] = 1											# temporary
		mask[mask == 0] = 255
		mask[mask == 1] = 0
		newimage = cv2.bitwise_and(newimage, newimage, mask=mask)
		newimage = cv2.bitwise_or(newimage, tempimg)
		
	return newimage         

Function for part-A:

In [5]:
def partA():
    imgs = []
            
    image1, pts1, tri1 = func_image1()
    image2, pts2, tri2 = func_image2()

    for t in range (1, 100):
        t = t/100
        npts = np.round(t*pts1 + (1-t)*pts2)
        n1 = intermediate_img(image1, npts, 1, pts1, pts2)
        n2 = intermediate_img(image2, npts, 2, pts1,  pts2)
        newImage = np.uint8(np.round((n1*t) + (n2*(1-t))))
        imgs.append(newImage)
    
    return imgs

Function for part-B:

In [9]:
def partB ():
    imgs = []
            
    image1, temp, tri1 = func_image1()
    image2, temp, tri2 = func_image2()

    file = open("k.txt", 'r')

    T = int(file.readline())
    pts1 = np.ones((T, 2), dtype='int32')
    pts2 = np.ones((T, 2), dtype='int32')

    data = file.readlines()
    i = 0                                   # i ~ counter for the no of points done
    for line in data:
        points = (line.split())
        pts1[i, 0] = points[0]
        pts1[i, 1] = points[1]
        pts2[i, 0] = points[2]
        pts2[i, 1] = points[3]
        i+=1

    for t in range (1, 100):
        t = t/100
        npts = np.round(t*pts1 + (1-t)*pts2)
        n1 = intermediate_img(image1, npts, 1, pts1, pts2)
        n2 = intermediate_img(image2, npts, 2, pts1,  pts2)
        newImage = np.uint8(np.round((n1*t) + (n2*(1-t))))
        imgs.append(newImage)

    return imgs

Main function:

In [10]:
# main function

choice = int(input("Enter choice for the parts:\n1 - part-A\n2 - part-B\n"))
if choice == 1:
    frames = partA()
    imageio.mimsave('result1.gif', frames, format='GIF', fps=5)
elif choice == 2:
    frames = partB()
    imageio.mimsave('result2.gif', frames, format='GIF', fps=5)
else:
    print("Invalid input!!")